Importing Libraries

In [1]:
!pip install ColorThief

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
from colorthief import ColorThief
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Input, BatchNormalization, Lambda, AveragePooling2D, ZeroPadding2D, Reshape, Activation
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

In [3]:
tf.test.is_built_with_cuda()

True

In [4]:
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [5]:
tf.config.list_physical_devices('GPU')

[]

In [6]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


EDA

TRAIN DATA

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
# Folder containing all the training data.
car_train = "/content/drive/MyDrive/cardata/Splited_dataset/train"
print('There are', len(os.listdir(car_train)),'folders in the training dataset')

There are 48 folders in the training dataset


In [34]:
mylist = []
for labels in os.listdir(car_train):
    for label in os.listdir(car_train+"/"+labels): 
        car_add = car_train+"/" + labels + '/' + label
        car_value = (labels, car_add)                
        if car_value not in mylist:
            mylist.append(car_value)
            
train_df = pd.DataFrame(np.array(mylist), columns= ["car", "image path"])

In [35]:
train_df.sample(10)

,car,image path
546,Honda_civic_2005,/content/drive/MyDrive/cardata/Splited_dataset...
173,Honda_City_aspire,/content/drive/MyDrive/cardata/Splited_dataset...
939,Suzuki_cultus_2018,/content/drive/MyDrive/cardata/Splited_dataset...
1042,Suzuki_cultus_2018,/content/drive/MyDrive/cardata/Splited_dataset...
751,Suzuki_alto_2007,/content/drive/MyDrive/cardata/Splited_dataset...
13,Daiatsu_Core,/content/drive/MyDrive/cardata/Splited_dataset...
2207,Toyota_corolla_2016,/content/drive/MyDrive/cardata/Splited_dataset...
2391,Toyota_corolla_2016,/content/drive/MyDrive/cardata/Splited_dataset...
1828,Suzuki_Mehran,/content/drive/MyDrive/cardata/Splited_dataset...
2338,Toyota_corolla_2016,/content/drive/MyDrive/cardata/Splited_dataset...


In [36]:
# Folder containng all the files for the test set. 
car_test= "/content/drive/MyDrive/cardata/Splited_dataset/test"
print('There are', len(os.listdir(car_test)),'folders in the test dataset')



There are 48 folders in the test dataset


In [37]:
# Create test_df
mylist2 = []
for labels in os.listdir(car_test):
    for label in os.listdir(car_test+"/"+labels): 
        car_add = car_test +"/"+ labels + '/' + label
        car_value = (labels, car_add)                
        if car_value not in mylist:
            mylist2.append(car_value)   

test_df = pd.DataFrame(np.array(mylist2), columns= ["car", "image path"])
test_df.sample(10)

,car,image path
318,Suzuki_cultus_2018,/content/drive/MyDrive/cardata/Splited_dataset...
501,Toyota_Aqua,/content/drive/MyDrive/cardata/Splited_dataset...
385,Suzuki_Mehran,/content/drive/MyDrive/cardata/Splited_dataset...
382,Suzuki_Mehran,/content/drive/MyDrive/cardata/Splited_dataset...
699,Toyota_Vitz,/content/drive/MyDrive/cardata/Splited_dataset...
287,Suzuki_cultus_2019,/content/drive/MyDrive/cardata/Splited_dataset...
612,Toyota_corolla_2016,/content/drive/MyDrive/cardata/Splited_dataset...
530,Toyota_corolla_2011,/content/drive/MyDrive/cardata/Splited_dataset...
20,Daiatsu_Core,/content/drive/MyDrive/cardata/Splited_dataset...
378,Suzuki_Mehran,/content/drive/MyDrive/cardata/Splited_dataset...


In [38]:
# Load in colors.csv and create df with index as color
clr_headers = ['color', 'full_name', 'hex', 'r', 'g', 'b'] # list for header
colors = pd.read_csv('/content/drive/MyDrive/colorsfinaldata.csv', names=clr_headers, index_col = 'color') # Load in csv
colors['rgb'] = list(zip(colors['r'], colors['g'], colors['b'])) # combine r/g/b as rgb tuple
colors_df = colors.drop(columns=['full_name', 'hex', 'r', 'g', 'b']) # drop old columns and save to new df
colors_dict = dict(zip(colors_df.index, colors_df['rgb'])) # create dictionary with color(key):rgb(value)

def classify_color(filepath):
    print(filepath)
    # Color thief will find the most dominant rgb color in an image
    color_thief = ColorThief(filepath)
    dominant_color = color_thief.get_color(quality=1)

    # Manhattan distance between "classification points"
    manhattan = lambda x,y : abs(x[0] - y[0]) + abs(x[1] - y[1]) + abs(x[2] - y[2]) 
    
    # Find the best color in dictionary and find Manhattan distance
    distances = {k: manhattan(v, dominant_color) for k, v in colors_dict.items()}
    
    # Minimum distance to color point
    color = min(distances, key=distances.get) 
    return color

In [39]:
 train_df['color'] = train_df['image path'].apply(classify_color)

/content/drive/MyDrive/cardata/Splited_dataset/train/Daiatsu_Core/11886.jpg
/content/drive/MyDrive/cardata/Splited_dataset/train/Daiatsu_Core/19726.jpg
/content/drive/MyDrive/cardata/Splited_dataset/train/Daiatsu_Core/20271.jpg
/content/drive/MyDrive/cardata/Splited_dataset/train/Daiatsu_Core/10184.jpg
/content/drive/MyDrive/cardata/Splited_dataset/train/Daiatsu_Core/18686.jpg
/content/drive/MyDrive/cardata/Splited_dataset/train/Daiatsu_Core/1576.jpg
/content/drive/MyDrive/cardata/Splited_dataset/train/Daiatsu_Core/1554.jpg
/content/drive/MyDrive/cardata/Splited_dataset/train/Daiatsu_Core/19724.jpg
/content/drive/MyDrive/cardata/Splited_dataset/train/Daiatsu_Core/163.jpg
/content/drive/MyDrive/cardata/Splited_dataset/train/Daiatsu_Core/14817.jpg
/content/drive/MyDrive/cardata/Splited_dataset/train/Daiatsu_Core/18354.jpg
/content/drive/MyDrive/cardata/Splited_dataset/train/Daiatsu_Core/17026.jpg
/content/drive/MyDrive/cardata/Splited_dataset/train/Daiatsu_Core/16445.jpg
/content/drive/M

In [40]:
train_df.tail()

,car,image path,color
3103,Toyota_Vitz,/content/drive/MyDrive/cardata/Splited_dataset...,onyx
3104,Toyota_Vitz,/content/drive/MyDrive/cardata/Splited_dataset...,dim_gray
3105,Toyota_Vitz,/content/drive/MyDrive/cardata/Splited_dataset...,medium_jungle_green
3106,Toyota_Vitz,/content/drive/MyDrive/cardata/Splited_dataset...,camouflage_green
3107,Toyota_Vitz,/content/drive/MyDrive/cardata/Splited_dataset...,black_olive


In [41]:
train_df.head()

,car,image path,color
0,Daiatsu_Core,/content/drive/MyDrive/cardata/Splited_dataset...,onyx
1,Daiatsu_Core,/content/drive/MyDrive/cardata/Splited_dataset...,black_olive
2,Daiatsu_Core,/content/drive/MyDrive/cardata/Splited_dataset...,black_olive
3,Daiatsu_Core,/content/drive/MyDrive/cardata/Splited_dataset...,black_leather_jacket
4,Daiatsu_Core,/content/drive/MyDrive/cardata/Splited_dataset...,dark_chestnut
